In [ ]:
# imports
import pandas as pd
import numpy as np
import wandb

In [ ]:
from data_handler import preprocessData, fitSurface

In [ ]:
paths = [
    "../data/processed/pca/predicted_iv16.csv",
    "../data/processed/pca/predicted_iv17.csv",
    "../data/processed/pca/predicted_iv18.csv",
    "../data/processed/pca/predicted_iv19.csv",
    "../data/processed/pca/predicted_iv20.csv",
    "../data/processed/pca/predicted_iv21.csv",
    "../data/processed/pca/predicted_iv22.csv"
]
df = pd.concat([pd.read_csv(path) for path in paths])
val_df = pd.read_csv("../data/processed/pca/predicted_iv23.csv")

In [ ]:
from models.vae_master import IVSFeatureExtractor

In [ ]:
extractor = IVSFeatureExtractor(
        hidden_dim=4096,
        latent_dim=16,
        beta=1.0,
        learning_rate=0.001
)
# 4096_64_1_001

In [ ]:
processed_data = extractor.prepare_data(df)

In [ ]:
extractor.train(processed_data, val_df, batch_size=256, n_epochs=4000)

In [ ]:
features = extractor.extract_features(processed_data)

In [ ]:
df_features = pd.DataFrame(features, columns=[f'feature_{i}' for i in range(features.shape[1])])

In [ ]:
df = pd.concat([df, df_features], axis=1)
print(df.head())

In [ ]:
df.to_csv('../data/processed/vae/features_vae_iv16_22_16.csv', index=False)

## LSTM

In [5]:
from models.lstm import CustomLSTMCell, CustomLSTMModel, ModelManager, DatasetManager

In [19]:
feature_cols = [f'feature_{i}' for i in range(16)]

In [16]:
import pandas as pd
vae_path = '../data/processed/vae/features_vae_iv16_22_16.csv'
df = pd.read_csv(vae_path)

In [17]:
df.head()

,date,tau,m,IV,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,2016-01-04,0.027397,-0.510826,0.326153,-0.028021,-0.018182,0.631262,-0.012937,0.010283,0.008706,-0.008162,0.005395,0.020689,0.025132,-0.035328,0.013297,0.292830,0.051068,0.001593,-0.004782
1,2016-01-04,0.027397,-0.223144,0.291228,0.094499,-0.021492,0.521984,-0.012051,0.016390,0.004015,-0.006669,0.006596,0.020506,0.025154,-0.021845,0.017183,0.205352,0.037030,0.004877,0.009910
2,2016-01-04,0.027397,-0.105361,0.286565,0.293737,-0.019346,0.761039,-0.007869,0.012253,0.004778,-0.013257,0.006969,0.025855,0.029640,-0.030230,0.015397,0.283865,0.054386,0.002579,0.005995
3,2016-01-04,0.027397,-0.051293,0.286299,-0.724118,-0.007250,0.172510,-0.001183,-0.000178,0.011536,0.002503,-0.006915,0.006890,0.017489,-0.017773,-0.002999,0.492829,0.019699,0.013059,-0.005291
4,2016-01-04,0.027397,-0.025318,0.286591,-0.568138,-0.008606,0.297375,-0.004034,0.000377,0.005638,-0.004817,-0.005480,0.009535,0.021787,-0.019562,-0.002383,0.628638,0.029114,0.017934,-0.001628


In [20]:
vae_path = '../data/processed/vae/features_vae_iv16_22_16.csv'
dataset = DatasetManager(vae_path)
features, targets = dataset.make_train_target_pairs(feature_cols)
print('Features shape:', features.shape)
print('Targets shape:', targets.shape)

Features shape: torch.Size([270864, 48])
Targets shape: torch.Size([270864, 16])


In [21]:
#Split the dataset into training and validation sets
# split = int(0.8 * len(features))
# train_features, val_features = features[:split], features[split:]
# train_targets, val_targets = targets[:split], targets[split:]

# no split
train_features, train_targets = features, targets

In [9]:
print('Initializing model...')
model_path = './ckpts/lstm_vae_1620_1024.pth'
model = ModelManager(input_dim=192, hidden_dim=1024, output_dim=64, model_path=model_path, learning_rate=0.01)
model.train(train_features, train_targets, epochs=200000)

#val_loader = DataLoader(TensorDataset(val_features, val_targets), batch_size=1, shuffle=False)
#model.validate(val_loader)

model.save_model()

Initializing model...


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abhigyanshanker (abx-group). Use `wandb login --relogin` to force relogin


Epoch 1 Loss: nan MAPE: nan
Epoch 2 Loss: nan MAPE: nan
Epoch 3 Loss: nan MAPE: nan


KeyboardInterrupt: 

In [22]:
import numpy as np

In [23]:
# print random feature-sample pair
print('Random feature-sample pair:')
idx = np.random.randint(0, len(train_features))
print('Feature:', train_features[idx])
print('Target:', train_targets[idx])

Random feature-sample pair:
Feature: tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
Target: tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])
